In [1]:
!gcloud config set project etsy-bigquery-adhoc-prod


If you have a compatible Python interpreter installed, you can use it by setting
the CLOUDSDK_PYTHON environment variable to point to it.

Updated property [core/project].


In [2]:
%load_ext google.cloud.bigquery

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import pandas as pd
import numpy as np

## Coverage in FBV2

```sql
select sum(queryLevelMetrics_gms)
from `etsy-ml-systems-prod.feature_bank_v2.query_feature_bank_2023-10-01`
where queryTaxoDemandFeatures_purchaseTopTaxonomyPaths is not null
and queryTaxoDemandFeatures_purchaseTopTaxonomyPaths.list[0] is not null
```

```sql
select sum(q.gms)
from `etsy-ml-systems-prod.feature_bank_v2.query_feature_bank_2023-10-01` fb
left join `etsy-data-warehouse-prod.rollups.query_level_metrics_raw` q
on fb.key = q.query_raw
```

## Coverage in a day of FL data

```sql
create or replace table `etsy-sr-etl-prod.yzhang.web_fl_train_1002`
as (
    with fl as (
      select
          requestUUID, visitId, position, 
          candidateInfo.docInfo.listingInfo.listingId, 
          candidateInfo.docInfo.listingInfo.verticaListings.taxonomyPath,
          ctx.docInfo.queryInfo.query,
          ctx.docInfo.queryInfo.queryTaxoDemandFeatures.*,
          attributions,
      from `etsy-ml-systems-prod.attributed_instance.query_pipeline_web_organic_2023_10_02`, 
          unnest(contextualInfo) as ctx
      where ctx.docInfo.queryInfo.query is not null
      order by requestUUID, position
    ),
    query_gms as (
      select query_raw, gms 
      from `etsy-data-warehouse-prod.rollups.query_level_metrics_raw` 
    )
    select
      fl.*, query_gms.gms
    from fl  
    left join query_gms 
    on fl.query = query_gms.query_raw
)
```

```sql
SELECT count(distinct query)
FROM `etsy-sr-etl-prod.yzhang.web_fl_train_1002`
where array_length(purchaseTopTaxonomyPaths) > 0
```

+ Queries with purchased taxonomies: 331904 / 925234 (36%)
+ Accounting for GMS 101571374484320 / 107290298625912 (94.7%)

+ Queries with clicked taxonomies purchase: 593278 / 925234 (64%)
+ accounting for GMS: 101857085524361 / 107290298625912 (94.9%)

## Coverage in 3 days of RPC data

```sql
with rpc_query as (
  select distinct query
  from `etsy-sr-etl-prod.yzhang.query_taxo_lastpass_rpc` rpc
  where purchase_top_paths is not null
  and purchase_top_paths.list[0] is not null
),
qgms as (
  select query, q.gms
  from rpc_query
  left join `etsy-data-warehouse-prod.rollups.query_level_metrics_raw` q
  on rpc_query.query = q.query_raw
)
select sum(gms) from qgms
```

## Is it possible listing is purchased but not in query top taxonomy? If yes how often?